In [ ]:
import pandas as pd
import sqlite3
import esco_utils as eu
import pandas.io.sql as psql

import psycopg2 as pg
import pandas.io.sql as psql

con = sqlite3.connect('data/cache.db')
esco_con = sqlite3.connect('data/esco_v1.1.0.db')

/opt/conda/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## Skill/Occupation graph  
In each language we have the following graph 
- _Concepts_: 
    - Skills. Concept types:
        - KnowledgeSkillCompetence: in `skills` table
        - skillGroup: in `skillGroup` table
    - Occupations. Concept types:
        - Occupation: in `occupations` table
        - occupationGroup: in `ISCOGroups` table
- _Edges_
    - non-hierarchical:
        - occupation-skill
            - essential: eg. software engineer (occ) -> programming (skill) is essential
            - optional: eg. software engineer (occ) -> linear algebra (skill) is optional
        - skill-skill
            - essential
            - optional
    - hierarchical
        - occupation -> broader occupation (parent)
        - skill -> broader skill (parent) 

# Summary of ESCO tables
- `jobs` database by Azuna. non-Arabic job vacancies, provided to us by SkillLab 
- `data/crawl.db`: Arabic jobs crawled by us from `forsana.com, bayt.com, fu1sa.com, adwhit.com`
- `data/cache.db`: ESCO tags, tasks, projects, and occupations and skills
    - `tags`: The tag table contains individual tags inserted by users.
    - `task_data`: Contains all tasks that are not cancelled. 
    - `projects` and `project_labelers`: project name and id, plus labelers in each project
    - Occupations:
        - `occupations`: contain title, alternative title, and description of all ISCO occupations in all EU languages. 
        - `ISCOGroups`: contain the groups of occupations, contains the tree taxonomy of occupations
    - Skills:
        - `skills`: skills data, for all languages (in `lang`) Skill types: 'skill/competence', 'knowledge', 'skill/competence knowledge'
        - `skillGroups`: skill groups for all languages (in `lang`)
    - Relations: 
        - `occupationSkillRelations`: relationships between skills and occupations, as a bipartite graph
        - `skillSkillRelations`: relations between skills as a graph
        
list of all ESCO-related tables:
- transversalSkillsCollection
- skillsHierarchy
- researchSkillsCollection
- skills
- occupations
- ictSkillsCollection
- greenSkillsCollection
- languageSkillsCollection
- ISCOGroups
- digCompSkillsCollection
- skillGroups
- conceptSchemes
- occupationSkillRelations
- broaderRelationsOccPillar
- skillSkillRelations
- broaderRelationsSkillPillar

# Jobs 
### Azuna
All Job postings _except Arabic_ are in a postgres DB running at `localhost:5678`, in table `jobs`. 
You can retrieve the database connection using the helper function `con, _ = eu.load_skillLab_DB()`. The important columns in this table are:
- `description`: the description of the job posting
- `title`: title of the job
- `description`: description about the job  
- `location_country`: country where job was posted, eg. `NL` for netherlands or `GB` for Great Britain 

In [13]:
jobs_con, _ = eu.load_skillLab_DB()

psql.read_sql(f"""
        SELECT title, description,location_country 
        FROM jobs 
        WHERE location_country='GB' 
        LIMIT 3
        """, jobs_con)

,title,description,location_country
0,Speech and Language Teaching Assistant Februar...,Graduate Speech & Language Teaching Assistant ...,GB
1,SEMH Teaching Assistant,Do you have the passion and talent for support...,GB
2,Intervention Teacher KS2 Sherwood,Intervention Teacher | KS2 | Full Time | Febru...,GB


### `fu1sa_proc`
`9915` jobs crawled from `fu1sa.com`. Columns: 
- `title` and `description`: title and description of the job 
- `date` and `place`: date and location the job is posted 
- `views`: how many times the job is viewed

In [15]:
crawl_db = sqlite3.connect('data/crawl.db')
pd.read_sql(f" SELECT * FROM fu1sa_proc LIMIT 3", crawl_db)

,post,title,views,date,place,description
0,24,اعلان موعد الاختبار التحريري للمتقدمين على وظا...,165,29/07/2019 - 08:12 AM,مدينة : الخرج,تعلن الهيئة العامة للأصاد وحماية البيئة عن تحد...
1,25,سدافكو توفّر وظائف شاغرة لحملة الثانوية العامة...,80,30/07/2019 - 09:52 AM,مدينة : الخرج,أعلنت الشركة السعودية سدافكو (حليب السعودية) ع...
2,27,اعلان وظائف على البنود في الشؤون الصحية بالقريات,69,31/07/2019 - 07:15 AM,مدينة : الخرج,أعلنت المديرية العامة للشؤون الصحية بمحافظة ال...


### `adwhit_proc`: 
`22594` jobs crawled from `adwhit.com`. Columns:
- `title` and `description`: the title and description of the job posting
- `meta`: unprocessed meta-data, containing date, company name, and place

In [16]:
crawl_db = sqlite3.connect('data/crawl.db')
pd.read_sql(f" SELECT * FROM adwhit_proc LIMIT 3", crawl_db)

,url,meta,title,description
0,/مطلوب-انسات-لوظيفة-مدخل-بيانات,الشركة المقدمة للعمل:\nadwhit\nتاريخ النشر: 18...,مطلوب انسات لوظيفة مدخل بيانات,"( الرجاء قراءة الاعلان كاملا )\nمطلوب انسات ""ح..."
1,/مطلوب-موظفة-كول-سنتر-لشركة-سياحية-في-تقسيم-اس...,الشركة المقدمة للعمل:\nRoyal mark group\nتاريخ...,مطلوب موظفة كول سنتر لشركة سياحية في تقسيم اسط...,السلام عليكم مطلوب موظفة كول سنتر لشركة سياحية...
2,/مطلوب-video-grapher-لشركة-في-اسطنبول-32240,الشركة المقدمة للعمل:\nAKD İNVEST\nتاريخ النشر...,مطلوب Video Grapher لشركة في اسطنبول,Nitelikler ve İş Tanımı\nÖnde gelen bir emlak ...


# Tasks and Tags

### `tags` table 
If anntator `A1` in response to `task1` confirms tags `1,2,3`, they appear as three rows.
important columns:
- `task_id`: the task 
- `occupation_id` & `occupation_title`: the id & title of the occupation tag chosen by user
- `inserted_at` & `label_time`: time for the each tag

In [17]:
pd.read_sql("SELECT * FROM tags LIMIT 1",con)

,index,inserted_at,label_time,labeler_id,lang,task_id,occupation_id,occupation_title
0,0,"Wed, 12 Jan 2022 14:04:25 GMT",8347,61dda30da33565ec1ce31aab,None,61dde40c527776b760a85fa0,1864,sales assistant


### `task_data` 
Important columns: 
- `project_id` and `project_name`: id & name of the project for this task
- `_id`: the task id 
- `title` and `description`: the title and description of the task. 

In [18]:
pd.read_sql("SELECT * FROM task_data LIMIT 1",con)

,project_name,project_id,_id,created_at,status,task-type,total_labels,updated_at,description,title
0,United Kingdom,61ddbe23527776b760a84bdd,61dddc3e527776b760a85c18,"Tue, 11 Jan 2022 19:36:30 GMT",pending,esco-text-tagging,0,"Tue, 11 Jan 2022 19:36:30 GMT",We're pleased to announce that SThree is looki...,Credit Controller - DACH


### `project` and `project_labelers`
get project name and id, plus labelers in each project

In [19]:
pd.read_sql("SELECT * FROM projects LIMIT 1",con)

,created_at,icon_id,labels_per_task,model_id,project_id,project_name,updated_at,lang,total_complete_tasks,total_labels,total_tasks,total_tasks_labeled
0,"Tue, 11 Jan 2022 14:14:51 GMT",61dd99d3527776b760a8497a,3,7693d764-72e5-11ec-83e4-0242c0a8d007,61dd90dbce271774889fbc02,GB,"Tue, 11 Jan 2022 14:14:51 GMT",en,0,1,300,0


In [20]:
pd.read_sql("SELECT * FROM project_labelers LIMIT 1",con)

,project_id,labeler_status,_id,email
0,61dd90dbce271774889fbc02,active_labelers,,ari@gmail.com


# Occupations 

### `occupations`: title, description, and ISCO codes in all languages
Important coluns:
- `occupation_id`: same occupation_id used in the `tags` table 
- `iscoGroup`: the group corresponding of this occupation, same as `code` in `ISCOGroups` table
- `preferredLabel`: title
- `altLabels`: alternative titles, seperated by `\n`
- `description`: description
- `lang`: the language for the record
- `conceptUri`: the unique identifier of this occupation  

In [21]:
lang = 'en'
pd.read_sql(f"SELECT * FROM occupations WHERE lang='{lang}'",esco_con)

,conceptType,conceptUri,iscoGroup,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,regulatedProfessionNote,scopeNote,definition,inScheme,description,code,lang,esco_version,occupation_id,external_id
0,Occupation,http://data.europa.eu/esco/occupation/00030d09...,2654,technical director,technical and operations director\nhead of tec...,None,released,2016-07-05T13:58:41Z,http://data.europa.eu/esco/regulated-professio...,None,None,http://data.europa.eu/esco/concept-scheme/occu...,Technical directors realise the artistic visio...,2654.1.7,en,v1.1.0,2.0,http://data.europa.eu/esco/occupation/00030d09...
1,Occupation,http://data.europa.eu/esco/occupation/000e93a3...,8121,metal drawing machine operator,metal drawing machine technician\nmetal drawin...,None,released,2016-07-05T17:09:43Z,http://data.europa.eu/esco/regulated-professio...,None,None,http://data.europa.eu/esco/concept-scheme/memb...,Metal drawing machine operators set up and ope...,8121.4,en,v1.1.0,3.0,http://data.europa.eu/esco/occupation/000e93a3...
2,Occupation,http://data.europa.eu/esco/occupation/0019b951...,7543,precision device inspector,inspector of precision instruments\nprecision ...,None,released,2016-07-06T09:21:20Z,http://data.europa.eu/esco/regulated-professio...,None,None,http://data.europa.eu/esco/concept-scheme/occu...,Precision device inspectors make sure precisio...,7543.10.3,en,v1.1.0,4.0,http://data.europa.eu/esco/occupation/0019b951...
3,Occupation,http://data.europa.eu/esco/occupation/0022f466...,3155,air traffic safety technician,air traffic safety electronics hardware specia...,None,released,2017-01-17T11:40:37Z,http://data.europa.eu/esco/regulated-professio...,None,None,http://data.europa.eu/esco/concept-scheme/memb...,Air traffic safety technicians provide technic...,3155.1,en,v1.1.0,5.0,http://data.europa.eu/esco/occupation/0022f466...
4,Occupation,http://data.europa.eu/esco/occupation/002da35b...,2431,hospitality revenue manager,hospitality revenues manager\nyield manager\nh...,None,released,2017-01-17T13:33:42Z,http://data.europa.eu/esco/regulated-professio...,None,None,http://data.europa.eu/esco/concept-scheme/memb...,Hospitality revenue managers maximise revenue ...,2431.9,en,v1.1.0,6.0,http://data.europa.eu/esco/occupation/002da35b...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3003,Occupation,http://data.europa.eu/esco/occupation/ff656b3a...,2120,demographer,demography research analyst\ndemography studie...,None,released,2021-12-21T13:41:38.024Z,http://data.europa.eu/esco/regulated-professio...,None,None,http://data.europa.eu/esco/concept-scheme/occu...,Demographers study a variety of parameters rel...,2120.2,en,v1.1.0,2939.0,http://data.europa.eu/esco/occupation/ff656b3a...
3004,Occupation,http://data.europa.eu/esco/occupation/ff8d4065...,9612,sorter labourer,grader\nyard labourer\nrecycler\nrecycling sit...,None,released,2021-12-08T20:18:37.546Z,http://data.europa.eu/esco/regulated-professio...,None,None,http://data.europa.eu/esco/concept-scheme/memb...,Sorter labourers sort recyclable materials and...,9612.2,en,v1.1.0,2940.0,http://data.europa.eu/esco/occupation/ff8d4065...
3005,Occupation,http://data.europa.eu/esco/occupation/ffa4dd5d...,5414,armoured car guard,armoured truck escort\ntruck escort\narmoured ...,None,released,2022-01-13T09:51:50.643Z,http://data.europa.eu/esco/regulated-professio...,None,None,http://data.europa.eu/esco/concept-scheme/occu...,Armoured car guards ensure the safe transporta...,5414.1.2,en,v1.1.0,2941.0,http://data.europa.eu/esco/occupation/ffa4dd5d...
3006,Occupation,http://data.europa.eu/esco/occupation/ffade2f4...,2422,civil service administrative officer,government administrative officer\nlocal autho...,None,released,2016-07-05T16:17:26Z,http://data.europa.eu/esco/regulated-professio...,None,None,http://data.europa.eu/esco/concept-scheme/occu...,Civil service administrative officers perform ...,2422.1,en,v1.1.0,2942.0,http://data.europa.eu/esco/occupation/ffade2f4...

### `ISCOGroups`: the groups of occupations 
Columns: 
- `conceptUri`: the unique identifier of this resource
- `code`: code of thiso group, corresponds to `iscoGroup` column in the `occupations` table
- `preferredLabel`: the title 
- `altLabels`: the alternative title

__Hierarchy__: the hierarchy of ISCO groups is preserved in their `code`. Example: ISCO group with code `22` is "Health professionals", which is the parent of group "Medical doctors" with code `221`, which is a parent of "Specialist medical practitioners" with code `2212". 

In [9]:
pd.read_sql(f"SELECT * FROM ISCOGroups WHERE lang='{lang}'",esco_con)

,conceptType,conceptUri,code,preferredLabel,status,altLabels,inScheme,description,lang,esco_version
0,ISCOGroup,http://data.europa.eu/esco/isco/C0,0,Armed forces occupations,released,None,http://data.europa.eu/esco/concept-scheme/occu...,Armed forces occupations include all jobs held...,en,v1.1.0
1,ISCOGroup,http://data.europa.eu/esco/isco/C01,1,Commissioned armed forces officers,released,None,http://data.europa.eu/esco/concept-scheme/isco...,Commissioned armed forces officers provide lea...,en,v1.1.0
2,ISCOGroup,http://data.europa.eu/esco/isco/C011,11,Commissioned armed forces officers,released,None,http://data.europa.eu/esco/concept-scheme/isco...,Commissioned armed forces officers provide lea...,en,v1.1.0
3,ISCOGroup,http://data.europa.eu/esco/isco/C0110,110,Commissioned armed forces officers,released,None,http://data.europa.eu/esco/concept-scheme/occu...,Commissioned armed forces officers provide lea...,en,v1.1.0
4,ISCOGroup,http://data.europa.eu/esco/isco/C02,2,Non-commissioned armed forces officers,released,None,http://data.europa.eu/esco/concept-scheme/occu...,Non-commissioned armed forces officers enforce...,en,v1.1.0
...,...,...,...,...,...,...,...,...,...,...
614,ISCOGroup,http://data.europa.eu/esco/isco/C9621,9621,"Messengers, package deliverers and luggage por...",released,None,http://data.europa.eu/esco/concept-scheme/isco...,"Messengers, package deliverers and luggage por...",en,v1.1.0
615,ISCOGroup,http://data.europa.eu/esco/isco/C9622,9622,Odd job persons,released,None,http://data.europa.eu/esco/concept-scheme/isco...,"Odd-job persons clean, paint and maintain buil...",en,v1.1.0
616,ISCOGroup,http://data.europa.eu/esco/isco/C9623,9623,Meter readers and vending-machine collectors,released,None,http://data.europa.eu/esco/concept-scheme/isco...,Meter readers and vending-machine collectors s...,en,v1.1.0
617,ISCOGroup,http://data.europa.eu/esco/isco/C9624,9624,Water and firewood collectors,released,None,http://data.europa.eu/esco/concept-scheme/isco...,Water and firewood collectors collect water an...,en,v1.1.0


# Skills
### `skills` table
There are `13502` skills in each of the EU languages. 
Important columns: 
- `conceptUri`: the unique identifier of this skill 
- `preferredLabel & altLabels, description`: main title, alternative titles, and description for this skill
- `skillType`: 'skill/competence', 'knowledge', or both 'skill/competence knowledge'
- `lang`: the language 

In [11]:
pd.read_sql(f"SELECT * FROM skills WHERE lang='{lang}'",esco_con)

,conceptType,conceptUri,skillType,reuseLevel,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,scopeNote,definition,inScheme,description,lang,esco_version
0,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/0005c151-5b5a...,skill/competence,sector-specific,manage musical staff,manage staff of music\ncoordinate duties of mu...,None,released,2016-12-20T17:43:43Z,None,None,http://data.europa.eu/esco/concept-scheme/skil...,Assign and manage staff tasks in areas such as...,en,v1.1.0
1,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/00064735-8fad...,skill/competence,occupation-specific,supervise correctional procedures,oversee prison procedures\nmanage correctional...,None,released,2016-12-20T20:17:49Z,None,None,http://data.europa.eu/esco/concept-scheme/memb...,Supervise the operations of a correctional fac...,en,v1.1.0
2,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/000709ed-2be5...,skill/competence,sector-specific,apply anti-oppressive practices,apply non-oppressive practices\napply an anti-...,None,released,2016-12-20T19:18:19Z,None,None,http://data.europa.eu/esco/concept-scheme/skil...,"Identify oppression in societies, economies, c...",en,v1.1.0
3,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/0007bdc2-dd15...,skill/competence,sector-specific,control compliance of railway vehicles regulat...,monitoring of compliance with railway vehicles...,None,released,2016-12-20T20:02:19Z,None,None,http://data.europa.eu/esco/concept-scheme/skil...,"Inspect rolling stock, components and systems ...",en,v1.1.0
4,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/00090cc1-1f27...,skill/competence,cross-sector,identify available services,establish available services\ndetermine rehabi...,None,released,2016-12-20T20:15:17Z,None,None,http://data.europa.eu/esco/concept-scheme/memb...,Identify the different services available for ...,en,v1.1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13886,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/ffef5eb3-a15e...,skill/competence,sector-specific,remediate healthcare user's occupational perfo...,restore healthcare user's occupational perform...,None,released,2016-12-20T19:25:53Z,None,None,http://data.europa.eu/esco/concept-scheme/memb...,"Remediate or restore the cognitive, sensorimot...",en,v1.1.0
13887,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/fff0b074-5a76...,skill/competence,sector-specific,install transport equipment lighting,install transport equipment illumination\nfix ...,None,released,2016-12-20T20:03:21Z,None,None,http://data.europa.eu/esco/concept-scheme/skil...,Install lighting elements in transport equipme...,en,v1.1.0
13888,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/fff0e2cd-d0bd...,knowledge,sector-specific,natural language processing,NLP,None,released,2016-08-04T15:19:37Z,None,None,http://data.europa.eu/esco/concept-scheme/skil...,The technologies which enable ICT devices to u...,en,v1.1.0
13889,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/fff5bc45-b506...,skill/competence,cross-sector,coordinate construction activities,reviewing construction progress\nconstruction ...,None,released,2016-12-20T18:22:35Z,None,None,http://data.europa.eu/esco/concept-scheme/skil...,Coordinate the activities of several construct...,en,v1.1.0


### `skillGroups`: groups of skills  

In [22]:
pd.read_sql(f"SELECT * FROM skillGroups WHERE lang='{lang}'",esco_con)

,conceptType,conceptUri,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,scopeNote,inScheme,description,code,lang,esco_version
0,SkillGroup,http://data.europa.eu/esco/isced-f/00,generic programmes and qualifications,None,None,released,None,None,http://data.europa.eu/esco/concept-scheme/isce...,Generic programmes and qualifications are thos...,None,en,v1.1.0
1,SkillGroup,http://data.europa.eu/esco/isced-f/000,generic programmes and qualifications not furt...,None,None,released,None,None,http://data.europa.eu/esco/concept-scheme/skil...,None,None,en,v1.1.0
2,SkillGroup,http://data.europa.eu/esco/isced-f/0000,generic programmes and qualifications not furt...,None,None,released,None,None,http://data.europa.eu/esco/concept-scheme/skil...,None,None,en,v1.1.0
3,SkillGroup,http://data.europa.eu/esco/isced-f/001,basic programmes and qualifications,None,None,released,None,None,http://data.europa.eu/esco/concept-scheme/skil...,None,None,en,v1.1.0
4,SkillGroup,http://data.europa.eu/esco/isced-f/0011,basic programmes and qualifications,None,None,released,None,None,http://data.europa.eu/esco/concept-scheme/isce...,Basic programmes and qualifications are design...,None,en,v1.1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,SkillGroup,http://data.europa.eu/esco/skill/T6.3,applying civic skills and competences,seeking opportunities for self-empowerment and...,None,released,2021-09-06T09:49:10.132Z,The skills and competences referred to under t...,http://data.europa.eu/esco/concept-scheme/skil...,Interact with citizens and engage in communiti...,T6.3,en,v1.1.0
639,SkillGroup,http://data.europa.eu/esco/skill/T6.4,applying cultural skills and competences,"appreciating and expressing cultural, artistic...",None,released,2021-08-30T16:54:02.907Z,Creativity and respecting diversity underpin t...,http://data.europa.eu/esco/concept-scheme/skil...,"Express and interpret ideas, experiences and e...",T6.4,en,v1.1.0
640,SkillGroup,http://data.europa.eu/esco/skill/T6.5,applying entrepreneurial and financial skills ...,financial and entrepreneurial literacy,None,released,2021-08-30T16:54:20.061Z,None,http://data.europa.eu/esco/concept-scheme/skil...,Manage own and others finances and resources. ...,T6.5,en,v1.1.0
641,SkillGroup,http://data.europa.eu/esco/skill/T6.6,applying general knowledge,general education\ngeneral knoweldge,None,released,2021-08-30T16:55:23.01Z,Skills and competences includes under this hea...,http://data.europa.eu/esco/concept-scheme/skil...,Access and apply basic knowledge related to sc...,T6.6,en,v1.1.0


# Relations

### `occupationSkillRelations`: occupation & skills graph
There are `114402` edges between `occupations` table and `skills`. Important columns: 
- `occupationUri`: corresponds to `conceptUri` in the occupation table
- `skillUri`: corresponds ot `conceptUri` in the skill table
- `relationType`: can be either 'essential' or 'optional'

In [23]:
pd.read_sql("SELECT * FROM occupationSkillRelations", esco_con)

,occupationUri,relationType,skillType,skillUri,esco_version
0,http://data.europa.eu/esco/occupation/00030d09...,essential,knowledge,http://data.europa.eu/esco/skill/fed5b267-73fa...,v1.0.3
1,http://data.europa.eu/esco/occupation/00030d09...,essential,skill/competence,http://data.europa.eu/esco/skill/05bc7677-5a64...,v1.0.3
2,http://data.europa.eu/esco/occupation/00030d09...,essential,skill/competence,http://data.europa.eu/esco/skill/271a36a0-bc7a...,v1.0.3
3,http://data.europa.eu/esco/occupation/00030d09...,essential,skill/competence,http://data.europa.eu/esco/skill/47ed1d37-971b...,v1.0.3
4,http://data.europa.eu/esco/occupation/00030d09...,essential,skill/competence,http://data.europa.eu/esco/skill/591dd514-735b...,v1.0.3
...,...,...,...,...,...
123847,http://data.europa.eu/esco/occupation/ffc292f1...,optional,skill/competence,http://data.europa.eu/esco/skill/df487cde-38d4...,v1.0.3
123848,http://data.europa.eu/esco/occupation/ffc292f1...,optional,skill/competence,http://data.europa.eu/esco/skill/e975b791-b488...,v1.0.3
123849,http://data.europa.eu/esco/occupation/ffc292f1...,optional,skill/competence,http://data.europa.eu/esco/skill/ed938748-60ae...,v1.0.3
123850,http://data.europa.eu/esco/occupation/ffc292f1...,optional,skill/competence,http://data.europa.eu/esco/skill/f30a7ea4-77dd...,v1.0.3


### `skillSkillRelations`: graph between skills 
There are `5971` edges between skills.

Important columns: 
- `originalSkillUri`: first skill URI, corresponding to `conceptUri` in the `skills` table
- `relatedSkillUri`: second skill URI, corresponding to `conceptUri` in the `skills` table
- `relationType`: relation between the two skills, can be 'optional' or 'essential'

In [24]:
pd.read_sql("SELECT * FROM skillSkillRelations", esco_con)

,originalSkillUri,originalSkillType,relationType,relatedSkillType,relatedSkillUri,esco_version
0,http://data.europa.eu/esco/skill/00064735-8fad...,knowledge,optional,knowledge,http://data.europa.eu/esco/skill/d4a0744a-508b...,v1.0.3
1,http://data.europa.eu/esco/skill/000bb1e4-89f0...,knowledge,optional,knowledge,http://data.europa.eu/esco/skill/b70ab677-5781...,v1.0.3
2,http://data.europa.eu/esco/skill/0023e7a5-43da...,knowledge,optional,knowledge,http://data.europa.eu/esco/skill/5753e2ca-8934...,v1.0.3
3,http://data.europa.eu/esco/skill/00298d97-3dc3...,knowledge,optional,knowledge,http://data.europa.eu/esco/skill/1ff53c92-370d...,v1.0.3
4,http://data.europa.eu/esco/skill/008a0e0d-7380...,knowledge,optional,knowledge,http://data.europa.eu/esco/skill/af0bbc31-b54d...,v1.0.3
...,...,...,...,...,...,...
5894,http://data.europa.eu/esco/skill/ffe198e3-3f51...,knowledge,optional,knowledge,http://data.europa.eu/esco/skill/2b281511-176e...,v1.0.3
5895,http://data.europa.eu/esco/skill/ffe8bbf9-0fec...,knowledge,optional,knowledge,http://data.europa.eu/esco/skill/5548401e-dbe9...,v1.0.3
5896,http://data.europa.eu/esco/skill/ffea25cf-8eab...,knowledge,optional,knowledge,http://data.europa.eu/esco/skill/152119f5-d0a2...,v1.0.3
5897,http://data.europa.eu/esco/skill/ffea25cf-8eab...,knowledge,optional,knowledge,http://data.europa.eu/esco/skill/3f391588-3ba2...,v1.0.3


### `broaderRelationsOccPillar`: occupation hierarchical edges
columns:
- ‍`conceptUri`: the identifier of the occupation 
- `broaderUri`: the identifier of the parent occupation 

In [25]:
pd.read_sql("SELECT * FROM broaderRelationsOccPillar",esco_con)

,conceptType,conceptUri,broaderType,broaderUri,esco_version
0,ISCOGroup,http://data.europa.eu/esco/isco/C01,ISCOGroup,http://data.europa.eu/esco/isco/C0,v1.0.3
1,ISCOGroup,http://data.europa.eu/esco/isco/C011,ISCOGroup,http://data.europa.eu/esco/isco/C01,v1.0.3
2,ISCOGroup,http://data.europa.eu/esco/isco/C0110,ISCOGroup,http://data.europa.eu/esco/isco/C011,v1.0.3
3,ISCOGroup,http://data.europa.eu/esco/isco/C02,ISCOGroup,http://data.europa.eu/esco/isco/C0,v1.0.3
4,ISCOGroup,http://data.europa.eu/esco/isco/C021,ISCOGroup,http://data.europa.eu/esco/isco/C02,v1.0.3
...,...,...,...,...,...
3612,Occupation,http://data.europa.eu/esco/occupation/ff2c1183...,Occupation,http://data.europa.eu/esco/occupation/4b05bef2...,v1.0.3
3613,Occupation,http://data.europa.eu/esco/occupation/ff3a164d...,Occupation,http://data.europa.eu/esco/occupation/14031d4a...,v1.0.3
3614,Occupation,http://data.europa.eu/esco/occupation/ff4c28e2...,Occupation,http://data.europa.eu/esco/occupation/4fef1907...,v1.0.3
3615,Occupation,http://data.europa.eu/esco/occupation/ff63fa99...,Occupation,http://data.europa.eu/esco/occupation/bcead4e9...,v1.0.3


### `broaderRelationsSkillPillar`: skill hierarchical edges 
- ‍`conceptUri`: the identifier of the skill 
- `broaderUri`: the identifier of the parent skill 

In [26]:
pd.read_sql("SELECT * FROM broaderRelationsSkillPillar",esco_con)

,conceptType,conceptUri,broaderType,broaderUri,esco_version
0,SkillGroup,http://data.europa.eu/esco/isced-f/00,SkillGroup,http://data.europa.eu/esco/skill/K,v1.0.3
1,SkillGroup,http://data.europa.eu/esco/isced-f/000,SkillGroup,http://data.europa.eu/esco/isced-f/00,v1.0.3
2,SkillGroup,http://data.europa.eu/esco/isced-f/0000,SkillGroup,http://data.europa.eu/esco/isced-f/000,v1.0.3
3,SkillGroup,http://data.europa.eu/esco/isced-f/001,SkillGroup,http://data.europa.eu/esco/isced-f/00,v1.0.3
4,SkillGroup,http://data.europa.eu/esco/isced-f/0011,SkillGroup,http://data.europa.eu/esco/isced-f/001,v1.0.3
...,...,...,...,...,...
20767,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/ffc1e455-ced2...,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/a71577b2-7550...,v1.0.3
20768,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/ffc836d3-1897...,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/435efc22-0dcb...,v1.0.3
20769,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/ffddfc7c-a9dd...,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/21d2f96d-35f7...,v1.0.3
20770,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/fff5bc45-b506...,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/75d8e5d9-bef3...,v1.0.3
